In [3]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  # For splitting the dataset
from sklearn.ensemble import RandomForestRegressor  # For the Random Forest regression model
import shap
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_log_error
import os

pd.set_option('display.expand_frame_repr', False)


In [4]:
def save_dataframe(df,file_name):
  from google.colab import drive
  drive.mount('/content/drive')
  file_path = "/content/drive/My Drive/"+file_name+".csv"
  df.to_csv(file_path, index=False)
  print(f"File saved successfully at: {file_path}")

def load_dataframe_from_drive(file_name='Train'):
    """
    Function to load a DataFrame from a Google Drive file path.
    """
    from google.colab import drive
    drive.mount('/content/drive')

    # Load the file from Google Drive
    file_path = "/content/drive/My Drive/"+file_name+".csv"  # Replace with the saved file path
    df = pd.read_csv(file_path)
    return df

def save_model(model,model_name):
  from google.colab import drive
  drive.mount('/content/drive')

  import joblib

  # Assuming 'rf_model' is your trained RandomForestRegressor
  model_path = '/content/drive/My Drive/'+model_name+'.pkl'
  joblib.dump(model, model_path)

  print(f"Model saved at {model_path}")

def load_model(model_name):
  import joblib

  model = joblib.load('/content/drive/My Drive/'+model_name+'.pkl')
  print("Model loaded successfully!")
  return model

def RMSLE(y_test, y_pred):
    '''
    RSMLE approximates the percent change
    '''
    return np.sqrt(np.mean((np.log(y_pred) - np.log(y_test))**2))

def RMSE(y_, y_pred_):
    '''
    RSME
    '''
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

def train_model(df,column_to_predict,test_size_value=0.3,random_state_value=42):
    '''
    Function to train a Random Forest regression model on a given DataFrame.
    '''
    print("train_model START")

    X = df.drop(columns=[column_to_predict])
    y = df[column_to_predict]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_value, random_state=random_state_value)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)


    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print("RMSE Baseline accuracy:", y_test.std())
    print("Train RMSE:", RMSE(y_train, y_train_pred))
    print("Test RMSE:", RMSE(y_test, y_test_pred))

    print("train_model END")

    return model,(X_train, X_test, y_train, y_test)

def generate_X_Y_params(df,column_to_predict,test_size_value=0.3,random_state_value=42):
    X = df.drop(columns=[column_to_predict])
    y = df[column_to_predict]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_value, random_state=random_state_value)
    return X_train, X_test, y_train, y_test

def list_unique_values(df, column_name):
    """
    Function to list all unique values in a given column of a DataFrame.

    Parameters:
    - df: pandas DataFrame
    - column_name: Name of the column to find unique values

    Returns:
    - A list of unique values in the specified column.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")

    unique_values = df[column_name].unique()
    return unique_values


# Function to calculate YearMade based on ModelID
def calc_YearMade(df, model_id):
    valid_years = df.loc[(df['ModelID'] == model_id) & (df['YearMade'] > 1000), 'YearMade']
    return int(valid_years.mean()) if not valid_years.empty else 1000  # Default if no valid years exist

def update_YearMade(df):
  print("update_YearMade START")
  # Update YearMade where it is 1000
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df.loc[df['YearMade'] == 1000, 'ModelID'].apply(lambda model_id: calc_YearMade(df, model_id))
  # Compute the mean excluding rows where YearMade == 1000
  mean_yearmade = df.loc[df['YearMade'] != 1000, 'YearMade'].mean()

  # Update all rows where YearMade == 1000 with the calculated mean
  df.loc[df['YearMade'] == 1000, 'YearMade'] = int(round(mean_yearmade))

  print("update_YearMade END")

  return df

def update_NaN_To_None_or_Unspecified(df):
    """
    Updates NaN values to 'None or Unspecified' in columns where the value
    'None or Unspecified' is already present.
    """
    print("update_NaN_To_None_or_Unspecified START")

    update_to = 'None or Unspecified'

    for col in df.columns:
        unique_values = list_unique_values(df, col)  # Get unique values for the column

        # Check if 'None or Unspecified' exists and if there are NaN values
        if update_to in unique_values and pd.isnull(unique_values).any():
            df[col] = df[col].fillna(update_to)  # Fill NaN with the specified value

    print("update_NaN_To_None_or_Unspecified END")

    return df

def update_MachineHoursCurrentMeter(df):
  print("update_MachineHoursCurrentMeter START")

  # Update the 'MachineHoursCurrentMeter' column: replace NaN or 0 with 0
  column_name = 'MachineHoursCurrentMeter'

  # Check if the column exists
  if column_name in df.columns:
      df[column_name] = df[column_name].fillna(0)  # Replace NaN with 0
      df[column_name] = df[column_name].replace(0, 0)  # Ensure 0 stays 0
      #print(f"Updated '{column_name}' column successfully!")
  #else:
      #print(f"Column '{column_name}' does not exist in the DataFrame.")

  print("update_MachineHoursCurrentMeter END")

  return df

def update_auctioneerID(df):
  print("update_auctioneerID START")

  # Assuming 'df' is your DataFrame
  # Fill null values in the 'auctioneerID' column with 100.0
  if 'auctioneerID' in df.columns:
      df['auctioneerID'] = df['auctioneerID'].fillna(100.0)
      #print("'auctioneerID' null values filled with 100.0 successfully!")
  #else:
  #    print("Column 'auctioneerID' does not exist in the DataFrame.")
  print("update_auctioneerID END")

  return df

def update_Enclosure(df):
  print("update_Enclosure START")

  # Replace null values in the 'Enclosure' column with 'N/A'
  if 'Enclosure' in df.columns:
      df['Enclosure'] = df['Enclosure'].fillna('None or Unspecified')

  print("update_Enclosure END")

  return df

def convert_date_columns(df, date_column):
    """
    Convert date column into multiple numeric features like year, month, day, etc.
    Handles potential issues like missing date_column or invalid data.
    """
    print("convert_date_columns START")

    # Ensure the date_column exists in the DataFrame
    if date_column not in df.columns:
        raise KeyError(f"Column '{date_column}' does not exist in the DataFrame.")

    # Convert the column to datetime
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')  # Handle invalid date strings

    # Check if any date values are invalid after conversion
    if df[date_column].isnull().all():
        raise ValueError(f"All values in '{date_column}' could not be converted to datetime.")

    # Extract date features
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['day'] = df[date_column].dt.day
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['is_weekend'] = df['day_of_week'] >= 5  # Saturday=5, Sunday=6

    # Drop the original date column
    df = df.drop(columns=[date_column])

    print("convert_date_columns END")

    return df

def encode_all_categories(df):
  '''
  Function to convert all string columns in a given DataFrame into categories.
  '''
  print("encode_all_categories START")
  for col in df.select_dtypes(['object']):
    df[col] = df[col].astype('category')

  for col in df.select_dtypes(['category']):
    df[col] = df[col].cat.codes

  print("encode_all_categories END")

  return df

def prepare_dataframe(df_pre:pd.DataFrame):
  '''04/02/2025 Start'''
  df_pre=update_YearMade(df_pre)
  print()
  df_pre=update_NaN_To_None_or_Unspecified(df_pre)
  print()

  df_pre=update_MachineHoursCurrentMeter(df_pre)
  print()

  df_pre=update_auctioneerID(df_pre)
  print()

  df_pre=update_Enclosure(df_pre)
  print()

  df_pre=convert_date_columns(df_pre,'saledate')
  print()

  df_pre=encode_all_categories(df_pre)
  '''04/02/2025 End'''

  return df_pre

def run_test(model):
  file_path = "/content/drive/My Drive/Valid.csv"  # Replace with the saved file path
  df_valid = pd.read_csv(file_path)
  df_valid['SalePrice']=None
  df_valid=prepare_dataframe(df_valid)
  X = df_valid.drop(columns=['SalePrice'])
  y = df_valid['SalePrice']
  y = model.predict(X)

  df_valid.loc[X.index, 'SalePrice'] = y

  file_path = "/content/drive/My Drive/output.csv"  # Save to Google Drive root
  df_valid[['SalesID', 'SalePrice']].to_csv(file_path, index=False)

  print(f"File saved successfully at: {file_path}")

def run_train():
  pd.set_option('display.expand_frame_repr', False)
  '''
  df=load_dataframe_from_drive()
  df = df.sample(n=10000, random_state=42)
  df=prepare_dataframe(df)
  model,xs=train_model(df=df,column_to_predict='SalePrice')
  save_dataframe(df,'04022025_df')
  save_model(model,'04022025_model')
  '''

  df=load_dataframe_from_drive('04022025_df')
  model=load_model('04022025_model')
  xs=generate_X_Y_params(df,'SalePrice')
  return model,xs



**RUN CELL**

In [11]:
print('Starting.....')
model,xs=run_train()
print('Testing...')
run_test(model)


Starting.....
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-10-f9bf011598fe>:17: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


update_YearMade START
update_YearMade END

update_NaN_To_None_or_Unspecified START
update_NaN_To_None_or_Unspecified END

update_MachineHoursCurrentMeter START
update_MachineHoursCurrentMeter END

update_auctioneerID START
update_auctioneerID END

update_Enclosure START
update_Enclosure END

convert_date_columns START
convert_date_columns END

encode_all_categories START
encode_all_categories END
train_model START
RMSE Baseline accuracy: 22932.4005340408
Train RMSE: 2636.4997091339187
Test RMSE: 7050.654203775603
train_model END
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File saved successfully at: /content/drive/My Drive/04022025_df.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved at /content/drive/My Drive/04022025_model.pkl
Testing...
update_YearMade START
update_YearMade END

update_NaN_To_None_or_Unspe

<ipython-input-10-f9bf011598fe>:261: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[54815.   66690.   35675.25 ... 10480.   12325.   21685.  ]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  df_valid.loc[X.index, 'SalePrice'] = y
